In [1]:
import os 
os.chdir('../')
import torch
from model import Model
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          BertConfig, BertForMaskedLM, BertTokenizer,
                          GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
                          OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,
                          DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

In [2]:
device = torch.device("cpu")

model_name_or_path = "microsoft/codebert-base"

MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
    'openai-gpt': (OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    'bert': (BertConfig, BertForMaskedLM, BertTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES['roberta'] 

config = RobertaConfig.from_pretrained(model_name_or_path, cache_dir=None, output_attentions=True)

config.num_labels=1
tokenizer = RobertaTokenizer.from_pretrained(model_name_or_path,
                                            do_lower_case=False,
                                            cache_dir=None)
block_size = 20

In [3]:
model = RobertaForSequenceClassification(config)
model=Model(model, config, tokenizer, None)
# load checkpoints 
checkpoint_prefix = "/home/fjiriges/CodeXGLUE/Code-Code/Defect-detection/code/saved_models/checkpoint-best-acc/model.bin"
model.load_state_dict(torch.load(checkpoint_prefix)) 
model = model.to(device)

In [4]:
class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_tokens,
                 input_ids,
                 idx,
                 label):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.idx=str(idx)
        self.label=label
        
def convert_examples_to_features(js, tokenizer, block_size=512):
    #source
    code=' '.join(js['func'].split())
    code_tokens=tokenizer.tokenize(code)[:block_size-2]
    source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = block_size - len(source_ids)
    source_ids+=[tokenizer.pad_token_id]*padding_length
    
    return InputFeatures(source_tokens,source_ids,js['idx'],js['target'])

In [14]:
js = {'func': 'def min(a,b): if a > b: return a else: return b', 'idx': 0, 'target': 1}

code_example = convert_examples_to_features(js, tokenizer, block_size=block_size)

input_ids = torch.tensor([code_example.input_ids])
input_labels = torch.tensor([code_example.label])
input_tokens = code_example.input_tokens
attention_mask=input_ids.ne(1)

In [15]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

# generate reference indices for each sample
token_reference = TokenReferenceBase(reference_token_idx=ref_token_id) # use padding_idx for roberta
seq_length = len(code_example.input_ids)
reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

In [16]:
lm_loss, logit, output_attentions = model(input_ids=input_ids, 
                                          attention_mask=attention_mask,
                                          labels=input_labels)
pred = logit.item()
pred_ind = round(pred)

print(lm_loss, logit)
print(pred, pred_ind)

tensor(0.5386, grad_fn=<NegBackward0>) tensor([[0.5836]], grad_fn=<SigmoidBackward0>)
0.5835870504379272 1


In [17]:
def predict(input_ids=input_ids, attention_mask=attention_mask, labels=input_labels):
    lm_loss, logit, output_attentions = model(input_ids=input_ids, 
                                          attention_mask=attention_mask,
                                          labels=input_labels)
    
    return logit

In [18]:
lig = LayerIntegratedGradients(predict, model.encoder.roberta.embeddings)

In [19]:
# compute attributions and approximation delta using layer integrated gradients
attributions_ig, delta = lig.attribute(input_ids, 
                                       reference_indices,
                                       return_convergence_delta=True)

In [20]:
attributions = attributions_ig.sum(dim=2).squeeze(0)
attributions = attributions / torch.norm(attributions)
attributions = attributions.cpu().detach().numpy()

In [21]:
vis_data_records_ig = []

vis_data_records_ig.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            pred_ind,
                            input_labels,
                            "Pos",
                            attributions.sum(),
                            code_example.input_tokens,
                            delta))

In [22]:
print('Visualize attributions based on Integrated Gradients')
_ = visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
tensor([1]),1 (0.58),Pos,1.62,"#s def Ġmin ( a , b ): Ġif Ġa Ġ> Ġb : Ġreturn Ġa Ġelse : Ġreturn Ġb #/s"
